In [2]:
import numpy as np
import nltk
import json
import pickle
import re
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer

In [3]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\S_pattnaik\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\S_pattnaik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
tokenized_words=[]
classes = []
doc = []
ignoring_words = ['?', '!']
data_file = open('Categorize_Data.json').read()
intents = json.loads(data_file)

In [6]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern) #tokenizing
        tokenized_words.extend(w)
        doc.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [7]:
lemmatizer = WordNetLemmatizer()
lemmatized_words = [lemmatizer.lemmatize(words.lower()) for words in tokenized_words if w not in ignoring_words] #lemmatization

In [9]:
lemmatized_words = sorted(list(set(lemmatized_words))) 
classes = sorted(list(set(classes)))

In [10]:
pickle.dump(lemmatized_words,open('lem_words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

Train the Model

In [15]:
training_data = []
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning) 
empty_array = [0] * len(classes)

for d in doc:
    bag_of_words = []

    pattern = d[0]

    pattern = [lemmatizer.lemmatize(word.lower()) for word in pattern]

    for w in lemmatized_words:

        bag_of_words.append(1) if w in pattern else bag_of_words.append(0)

    output_row = list(empty_array)

    output_row[classes.index(d[1])] = 1

    training_data.append([bag_of_words, output_row])

random.shuffle(training_data)

training = np.array(training_data)

train_x = list(training[:,0])

train_y = list(training[:,1])

In [17]:
bot_model = Sequential()
bot_model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
bot_model.add(Dropout(0.5))
bot_model.add(Dense(64, activation='relu'))
bot_model.add(Dropout(0.5))
bot_model.add(Dropout(0.25))
bot_model.add(Dense(len(train_y[0]), activation='softmax'))

In [18]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
bot_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

C:\Anaconda\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [19]:
train_x = np.array(train_x)
train_y = np.array(train_y)
hist = bot_model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

Epoch 1/200
4/4 [==============================] - 1s 2ms/step - loss: 1.3995 - accuracy: 0.2105
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 1.4027 - accuracy: 0.2632
Epoch 3/200
4/4 [==============================] - 0s 2ms/step - loss: 1.3442 - accuracy: 0.2632
Epoch 4/200
4/4 [==============================] - 0s 3ms/step - loss: 1.2689 - accuracy: 0.5263
Epoch 5/200
4/4 [==============================] - 0s 3ms/step - loss: 1.2714 - accuracy: 0.5263
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 1.1589 - accuracy: 0.5789
Epoch 7/200
4/4 [==============================] - 0s 3ms/step - loss: 1.1673 - accuracy: 0.5263
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 1.1061 - accuracy: 0.5263
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.8852 - accuracy: 0.8947
Epoch 10/200
4/4 [==============================] - 0s 2ms/step - loss: 0.8855 - accuracy: 0.7895
Epoch 11/200
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.0569 - accuracy: 1.0000
Epoch 85/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0272 - accuracy: 1.0000
Epoch 86/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0497 - accuracy: 1.0000
Epoch 87/200
4/4 [==============================] - 0s 1000us/step - loss: 0.0152 - accuracy: 1.0000
Epoch 88/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0727 - accuracy: 0.9474
Epoch 89/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0689 - accuracy: 0.9474
Epoch 90/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1160 - accuracy: 0.9474
Epoch 91/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0475 - accuracy: 1.0000
Epoch 92/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0227 - accuracy: 1.0000
Epoch 93/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0469 - accuracy: 1.0000
Epoch 94/200
4/4 [============

4/4 [==============================] - 0s 1ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 167/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 168/200
4/4 [==============================] - 0s 3ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 169/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0067 - accuracy: 1.0000
Epoch 170/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0057 - accuracy: 1.0000
Epoch 171/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0151 - accuracy: 1.0000
Epoch 172/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0388 - accuracy: 1.0000
Epoch 173/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0085 - accuracy: 1.0000
Epoch 174/200
4/4 [==============================] - 0s 2ms/step - loss: 0.0364 - accuracy: 1.0000
Epoch 175/200
4/4 [==============================] - 0s 1ms/step - loss: 0.0169 - accuracy: 1.0000
Epoch 176/200
4/4 [=====

In [20]:
bot_model.save('chatbot_model.h5', hist)

Testing the Model

In [21]:
import pickle
import numpy as np
import json
from keras.models import load_model
import random
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [22]:
intents_file = json.loads(open('Categorize_Data.json').read())
lem_words = pickle.load(open('lem_words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
bot_model = load_model('chatbot_model.h5')

In [23]:
def cleaning(text):
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word.lower()) for word in words]
    return words

In [24]:
def bag_ow(text, words, show_details=True):
    sentence_words = cleaning(text)
    bag_of_words = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag_of_words[i] = 1
    return (np.array(bag_of_words))

In [30]:
def class_prediction(sentence, model):
    p = bag_ow(sentence, lem_words,show_details=False)
    result = bot_model.predict(np.array([p]))[0]
    ER_THRESHOLD = 0.30
    f_results = [[i,r] for i,r in enumerate(result) if r > ER_THRESHOLD]
    f_results.sort(key=lambda x: x[1], reverse=True)
    intent_prob_list = []
    for i in f_results:
        intent_prob_list.append({"intent": pred_class[i[0]], "probability": str(i[1])})
    return intent_prob_list

In [31]:
def getbotResponse(ints, intents):
    tag = ints[0]['intent']
    intents_list = intents['intents']
    for intent in intents_list:
        if(intent['tag']== tag):
            result = random.choice(intent['responses'])
            break
    return result
def bot_response(text):
    ints = class_prediction(text, bot_model)
    response = getbotResponse(ints, intents)
    return response

In [32]:
for i in range(3):
    text = input("You : ")
    print("Bot : ",bot_response(text))

You : Hi
1/1 [==============================] - 0s 16ms/step


NameError: name 'pred_class' is not defined